In [63]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import timedelta, date
import matplotlib.dates as mdates
import math
import warnings
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import accuracy_score
from sklearn import neighbors

warnings.filterwarnings('ignore')

#Imports necesarios

In [8]:
#Primero importamos el dataframe de entrenamiento
train = pd.read_csv('/home/matias/Escritorio/ML/trainPreProcess.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196414 entries, 0 to 196413
Data columns (total 17 columns):
Unnamed: 0                  196414 non-null int64
id                          196414 non-null object
created_on                  196414 non-null object
property_type_encoded       196414 non-null int64
place_name_encoded          196414 non-null int64
state_name_encoded          196414 non-null int64
lat                         196414 non-null float64
lon                         196414 non-null float64
surface_covered_in_m2       196414 non-null float64
surface_total_in_m2         196414 non-null float64
floor                       196414 non-null float64
rooms                       196414 non-null float64
price                       196414 non-null float64
distancia_obelisco          196414 non-null float64
distancia_minima_subte      196414 non-null float64
distancia_minima_privado    196414 non-null float64
distancia_minima_publico    196414 non-null float64
dtypes: float64

In [10]:
#Separemos el dataframe en 2, uno para realizar croosvalidation y el otro para entrenar
train1 = train[0:180000]
cross = train[180000:]
cross1 = train[180000:]
cross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16414 entries, 180000 to 196413
Data columns (total 17 columns):
Unnamed: 0                  16414 non-null int64
id                          16414 non-null object
created_on                  16414 non-null object
property_type_encoded       16414 non-null int64
place_name_encoded          16414 non-null int64
state_name_encoded          16414 non-null int64
lat                         16414 non-null float64
lon                         16414 non-null float64
surface_covered_in_m2       16414 non-null float64
surface_total_in_m2         16414 non-null float64
floor                       16414 non-null float64
rooms                       16414 non-null float64
price                       16414 non-null float64
distancia_obelisco          16414 non-null float64
distancia_minima_subte      16414 non-null float64
distancia_minima_privado    16414 non-null float64
distancia_minima_publico    16414 non-null float64
dtypes: float64(11), int64(4

In [12]:
#Nos quedamos solo con la latitud y longitud de los barrios
train2 = train1[['lat','lon','place_name_encoded']]
cross2 = cross[['lat','lon']]
crossValidation = cross1[['lat','lon','place_name_encoded']]
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 3 columns):
lat                   180000 non-null float64
lon                   180000 non-null float64
place_name_encoded    180000 non-null int64
dtypes: float64(2), int64(1)
memory usage: 4.1 MB


In [14]:
# Import KNN
from sklearn.neighbors import KNeighborsClassifier

In [15]:
knn = KNeighborsClassifier(n_neighbors=10)

In [16]:
print knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')


In [18]:
trainX = train2[['lat','lon']]
trainY = train2[['place_name_encoded']]
trainY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 1 columns):
place_name_encoded    180000 non-null int64
dtypes: int64(1)
memory usage: 1.4 MB


In [19]:
knn.fit(trainX,trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [20]:
predictions = knn.predict(cross2)
print predictions

[757 711 536 ..., 549 549 757]


In [21]:
crossValidation['prediciones'] = predictions
crossValidation.head()

,lat,lon,place_name_encoded,prediciones
180000,-34.442442,-58.860950,757,757
180001,-34.402644,-58.648236,711,711
180002,-34.432240,-58.819492,536,536
180003,-34.622481,-58.843267,230,230
180004,-34.608786,-58.946258,453,453


In [24]:
score = mean_squared_error(crossValidation['place_name_encoded'], predictions)
score

33127.049226270254

In [45]:
k_trials = [151, 152, 153, 155,149]
            
weights = ['uniform', 'distance']

algorithms = ['kd_tree', 'ball_tree']

predictionList = []
scoreList = []

for k in k_trials:
    for w in weights:
        for a in algorithms:
            knn = KNeighborsClassifier(n_neighbors=k, weights=w,algorithm=a)
            predictions = knn.fit(trainX, trainY).predict(cross2)
            predictions = predictions.flatten()
            #Creo archivo para submit
            predictionList.append('KNN-K'+str(k)+'-'+w+'-'+a)
            score =accuracy_score(crossValidation['place_name_encoded'], predictions)
            scoreList.append(score)

In [46]:
print scoreList

[0.81333008407457053, 0.81326916047276721, 0.85963202144510786, 0.85969294504691118, 0.81241623004752039, 0.81241623004752039, 0.85938832703789447, 0.85944925063969779, 0.81247715364932371, 0.81247715364932371, 0.85914463263068108, 0.85914463263068108, 0.81235530644571707, 0.81265992445473378, 0.85902278542707444, 0.85908370902887776, 0.81326916047276721, 0.81339100767637385, 0.8597538686487145, 0.85981479225051782]


In [48]:
resultados = pd.DataFrame()
resultados['parametros'] = predictionList
resultados['score'] = scoreList
rOrder = resultados.sort_values('score',ascending= False)
rOrder.head()


,parametros,score
19,KNN-K149-distance-ball_tree,0.859815
18,KNN-K149-distance-kd_tree,0.859754
3,KNN-K151-distance-ball_tree,0.859693
2,KNN-K151-distance-kd_tree,0.859632
7,KNN-K152-distance-ball_tree,0.859449


In [ ]:
#Dejamos esto un rato

# Lasso

In [49]:
#Primero importamos el dataframe de entrenamiento
train = pd.read_csv('/home/matias/Escritorio/ML/trainPreProcess.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196414 entries, 0 to 196413
Data columns (total 17 columns):
Unnamed: 0                  196414 non-null int64
id                          196414 non-null object
created_on                  196414 non-null object
property_type_encoded       196414 non-null int64
place_name_encoded          196414 non-null int64
state_name_encoded          196414 non-null int64
lat                         196414 non-null float64
lon                         196414 non-null float64
surface_covered_in_m2       196414 non-null float64
surface_total_in_m2         196414 non-null float64
floor                       196414 non-null float64
rooms                       196414 non-null float64
price                       196414 non-null float64
distancia_obelisco          196414 non-null float64
distancia_minima_subte      196414 non-null float64
distancia_minima_privado    196414 non-null float64
distancia_minima_publico    196414 non-null float64
dtypes: float64

In [52]:
train2 =train[['property_type_encoded','place_name_encoded','state_name_encoded','lat','lon','surface_covered_in_m2','surface_total_in_m2','floor','rooms','price','distancia_obelisco','distancia_minima_subte','distancia_minima_publico','distancia_minima_privado']]
train3 = train2[0:180000]
cross = train2[180000:]
cross2 = cross[['property_type_encoded','place_name_encoded','state_name_encoded','lat','lon','surface_covered_in_m2','surface_total_in_m2','floor','rooms','distancia_obelisco','distancia_minima_subte','distancia_minima_publico','distancia_minima_privado']]
crossValidation = cross[['property_type_encoded','place_name_encoded','state_name_encoded','lat','lon','surface_covered_in_m2','surface_total_in_m2','floor','rooms','price','distancia_obelisco','distancia_minima_subte','distancia_minima_publico','distancia_minima_privado']]
crossValidation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16414 entries, 180000 to 196413
Data columns (total 14 columns):
property_type_encoded       16414 non-null int64
place_name_encoded          16414 non-null int64
state_name_encoded          16414 non-null int64
lat                         16414 non-null float64
lon                         16414 non-null float64
surface_covered_in_m2       16414 non-null float64
surface_total_in_m2         16414 non-null float64
floor                       16414 non-null float64
rooms                       16414 non-null float64
price                       16414 non-null float64
distancia_obelisco          16414 non-null float64
distancia_minima_subte      16414 non-null float64
distancia_minima_publico    16414 non-null float64
distancia_minima_privado    16414 non-null float64
dtypes: float64(11), int64(3)
memory usage: 1.8 MB


In [53]:
trainLX = train3[['property_type_encoded','place_name_encoded','state_name_encoded','lat','lon','surface_covered_in_m2','surface_total_in_m2','floor','rooms','distancia_obelisco','distancia_minima_subte','distancia_minima_publico','distancia_minima_privado']]
trainLY = train3[['price']]
trainLY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 1 columns):
price    180000 non-null float64
dtypes: float64(1)
memory usage: 1.4 MB


In [54]:
from sklearn import linear_model
lasso= linear_model.Lasso(alpha = 0.1)
lasso.fit(trainLX,trainLY)
predictions = lasso.predict(cross2)
print(predictions)


[ 315167.02856146  304104.83127752  313637.38695322 ...,  371069.20273539
  274788.60751895  376797.69664081]


In [56]:
a_trials = [0.1, 0.2, 0.3,0.45,0.66,0.88]
            
predictionList = []
scoreList = []

for a in a_trials:
    lasso= linear_model.Lasso(alpha = a)
    predictions = lasso.fit(trainLX, trainLY).predict(cross2)
    predictions = predictions.flatten()
    predictionList.append('lasso'+' '+str(a))
    score =mean_squared_error(crossValidation['price'], predictions)
    scoreList.append(score)

In [61]:
resultados = pd.DataFrame()
resultados['parametros'] = predictionList
resultados['score'] = scoreList
rOrder = resultados.sort_values('score',ascending= False)
print(scoreList)

[1255336646994.3347, 1255336703614.8772, 1255336760236.0403, 1255336845168.9478, 1255336964077.3621, 1255337088651.0144]


In [64]:
k_trials = [1, 2, 3, 4, 5, 11, 21, 31, 51, 71, 91, 101, 151, 201, 251]
            
weights = ['uniform', 'distance']

algorithms = ['kd_tree', 'ball_tree']

predictionList = []
scoreList = []

for k in k_trials:
    for w in weights:
        for a in algorithms:
            knn = neighbors.KNeighborsRegressor(k, weights=w, algorithm=a)
            predictions = knn.fit(trainLX, trainLY).predict(cross2)
            predictions = predictions.flatten()
            predictionList.append('KNN-K'+str(k)+'-'+w+'-'+a)
            score = mean_squared_error(crossValidation['price'].values, predictions)
            scoreList.append(score)

In [67]:
resultados = pd.DataFrame()
resultados['parametros'] = predictionList
resultados['score'] = scoreList
rOrder = resultados.sort_values('score',ascending= True)
rOrder.head(20)

,parametros,score
17,KNN-K5-uniform-ball_tree,1.047778e+12
16,KNN-K5-uniform-kd_tree,1.048931e+12
20,KNN-K11-uniform-kd_tree,1.115877e+12
21,KNN-K11-uniform-ball_tree,1.116197e+12
59,KNN-K251-distance-ball_tree,1.173916e+12
58,KNN-K251-distance-kd_tree,1.173921e+12
55,KNN-K201-distance-ball_tree,1.174524e+12
54,KNN-K201-distance-kd_tree,1.174529e+12
50,KNN-K151-distance-kd_tree,1.178054e+12
51,KNN-K151-distance-ball_tree,1.178088e+12
